In [9]:
#Importieren der Pakete für Solver, Investitionsentscheidung
from oemof.tools import economics
from oemof import solph
#Datenanalyse
import pandas as pd

In [10]:
#Einlesen der Datenbasis --- normierte Profile/Verbraucher/Einspeisungen/Erzeuger/Bezüge/Energieumwandler/Speicher
profile = pd.read_excel("input_data.xlsx", sheet_name = "norm_profile")
sink = pd.read_excel("input_data.xlsx", sheet_name = "sink")
source = pd.read_excel("input_data.xlsx", sheet_name = "source")
transformer = pd.read_excel("input_data.xlsx", sheet_name = "transformer")
storage = pd.read_excel("input_data.xlsx", sheet_name = "storage")

In [14]:
#Modelldefinition im Zeitraum 2030 in stdl. Auflösung
Zeit = pd.date_range("1/1/2030", periods=8760, freq="H")

#Modelldefinition
Industrie = solph.EnergySystem(timeindex=Zeit)

#Gewichteter Kapitalkostensatz//Weighted Average Cost of Capital
r=0.05

#Definition der Energieträger bzw. Knotenpunkte als Bus
bel_ind = solph.Bus(label = "Strom Industrie")
bel_pv_ind = solph.Bus(label = "Strom PV")
bel_chp_ind = solph.Bus(label = "Strom KWK")
bh2_ind = solph.Bus(label = "H2 Industrie")
bhps_ind = solph.Bus(label = "HD-Dampf Industrie")
bth_h_ind = solph.Bus(label = "Prozesswaerme Industrie")
bth_l_ind = solph.Bus(label = "NT-Waerme Industrie")
btes_ind = solph.Bus(label = "WÜ-Verbindung Industrie")

#Hinzufügen der Busses zum Energiesystem
Industrie.add(bel_ind, bel_pv_ind, bel_chp_ind, bh2_ind, bhps_ind, bth_h_ind, bth_l_ind, btes_ind)

###SENKEN -- SINKS
#Strom- und Wärmelastgänge sowie Abwärmesenken
el_cons_ind = solph.Sink(
    label="Stromlast IND",
    inputs={bel_ind: solph.Flow(fix= profile.i0_el, nominal_value= sink.bedarf.iloc[2])})

th_l_cons_ind = solph.Sink(
    label="NT-Waerme IND",
    inputs={bth_l_ind: solph.Flow(fix= profile.nt1_th, nominal_value= sink.bedarf.iloc[3])})

th_h_cons_ind = solph.Sink(
    label="HT-Waerme IND",
    inputs={bth_h_ind: solph.Flow(fix= profile.ht1_th, nominal_value = sink.bedarf.iloc[4])})

th_l_sink_ind = solph.Sink(
    label="NT-Wärmesenke",
    inputs={bth_l_ind: solph.Flow()})

th_h_sink_ind = solph.Sink(
    label="HT-Wärmesenke",
    inputs={bth_h_ind: solph.Flow()})

#Netzeinspeisungen Strom
pv_exp_ind = solph.Sink(
        label="Stromexport PV IND",
        inputs={bel_pv_ind: solph.Flow(variable_costs = -1*sink.var_kosten.iloc[0])})


chp_exp_ind = solph.Sink(
        label="Stromexport KWK IND",
        inputs={bel_chp_ind: solph.Flow(variable_costs = -1*sink.var_kosten.iloc[1])})

###QUELLEN -- SOURCES
#EE-Erzeugung
pv_ind = solph.Source(
        label="PV IND",
        outputs={bel_pv_ind: solph.Flow(fix= profile.pv,
        investment = solph.Investment(
        ep_costs = economics.annuity(capex= source.invest.iloc[3], n=source.ANF.iloc[3], wacc=r),
        maximum = source.maximum.iloc[3]))})

#Netzbezüge Strom/Wasserstoff/Wärme
el_imp_ind = solph.Source(
        label="Stromimport Netz IND",
        outputs={bel_ind: solph.Flow(variable_costs = source.var_kosten.iloc[2],
        investment = solph.Investment(ep_costs = source.invest.iloc[2]))})

h2_imp_ind = solph.Source(
        label="H2-Import IND",
        outputs={bh2_ind: solph.Flow(variable_costs = source.var_kosten.iloc[0],
        investment = solph.Investment(ep_costs = source.invest.iloc[0]))})

th_imp_ind = solph.Source(
        label="Import Waermenetz IND",
        outputs={bth_l_ind: solph.Flow(variable_costs = source.var_kosten.iloc[1],
        investment = solph.Investment(ep_costs = source.invest.iloc[1]))})

##ENERGIEWANDLER -- TRANSFORMER
wp_ind = solph.Transformer(
        label='Waermepumpe IND',
        inputs={bel_ind: solph.Flow()},
        outputs={bth_l_ind: solph.Flow(
            investment=solph.Investment(
                ep_costs=economics.annuity(capex= transformer.invest.iloc[0], n= transformer.ANF.iloc[0], wacc= r),
                maximum = transformer.maximum.iloc[0],
                minimum = transformer.minimum.iloc[0]))},
            conversion_factors={bth_l_ind: transformer.η_th.iloc[0]})

elektrolyser_ind = solph.Transformer(
        label='Elektrolyseur IND',
        inputs={bel_ind: solph.Flow()},
        outputs={bh2_ind: solph.Flow(
            investment=solph.Investment(
                ep_costs=economics.annuity(capex= transformer.invest.iloc[1], n= transformer.ANF.iloc[1], wacc= r),
                maximum = transformer.maximum.iloc[1],
                minimum = transformer.minimum.iloc[1])), bth_l_ind: solph.Flow()},
            conversion_factors={bh2_ind: transformer.η_h2.iloc[1], bth_l_ind: transformer.η_th.iloc[1]})

    #Gas- und Dampfkraftwerk
gt_chp_ind = solph.ExtractionTurbineCHP(
        label='GT-KWK IND',
        inputs={bh2_ind: solph.Flow()},
        outputs={bel_chp_ind: solph.Flow(
            investment=solph.Investment(
                ep_costs=economics.annuity(capex= transformer.invest.iloc[2], n= transformer.ANF.iloc[2], wacc= r),
                maximum = transformer.maximum.iloc[2],
                minimum = transformer.minimum.iloc[2])), bhps_ind: solph.Flow()},
            conversion_factors={bel_chp_ind: transformer.η_el.iloc[2], bhps_ind: transformer.η_th.iloc[2]},
            conversion_factor_full_condensation={bel_chp_ind: transformer.η_el_full.iloc[2]})

st_ind = solph.Transformer(
        label='Dampfturbine IND', ##Dampfturbine zur Senkung des HT-Dampfes - Verstromung und Nutzung der NT-Wärme
        inputs={bhps_ind: solph.Flow()},
        outputs={bel_chp_ind: solph.Flow(
            investment=solph.Investment(
                ep_costs=economics.annuity(capex= transformer.invest.iloc[3],n= transformer.ANF.iloc[3], wacc= r),
                maximum = transformer.maximum.iloc[3],
                minimum = transformer.minimum.iloc[3])), bth_l_ind: solph.Flow()},
            conversion_factors={bel_chp_ind: transformer.η_el.iloc[3], bth_l_ind: transformer.η_th.iloc[3]})

pth_ind = solph.Transformer(
        label='Power-to-Heat IND',
        inputs={bel_ind: solph.Flow()},
        outputs={bth_h_ind: solph.Flow(
            investment=solph.Investment(
                ep_costs=economics.annuity(capex= transformer.invest.iloc[4],n= transformer.ANF.iloc[4], wacc= r),
                maximum = transformer.maximum.iloc[4],
                minimum = transformer.minimum.iloc[4]))},
            conversion_factors={bth_h_ind: transformer.η_th.iloc[4]})

bz_ind = solph.Transformer(
        label='Brennstoffzelle IND',
        inputs={bh2_ind: solph.Flow()},
        outputs={bel_chp_ind: solph.Flow(
            investment=solph.Investment(
                ep_costs=economics.annuity(capex= transformer.invest.iloc[5],n= transformer.ANF.iloc[5], wacc= r),
                maximum = transformer.maximum.iloc[5],
                minimum = transformer.minimum.iloc[5])), bth_l_ind: solph.Flow()}, 
            conversion_factors={bel_chp_ind: transformer.η_el.iloc[5], bth_l_ind: transformer.η_th.iloc[5]})

m_chp_ind = solph.Transformer(
        label='BHKW IND',
        inputs={bh2_ind: solph.Flow()},
        outputs={bel_chp_ind: solph.Flow(
            investment=solph.Investment(
                ep_costs=economics.annuity(capex= transformer.invest.iloc[6],n= transformer.ANF.iloc[6], wacc= r),
                maximum = transformer.maximum.iloc[6],
                minimum = transformer.minimum.iloc[6])), bth_l_ind: solph.Flow()},
            conversion_factors={bel_chp_ind: transformer.η_el.iloc[6], bth_l_ind: transformer.η_th.iloc[6]})

##SPEICHER - Storages
el_stor_ind = solph.components.GenericStorage(
        label="Stromspeicher IND",
        inputs={bel_ind: solph.Flow()},
        outputs={bel_ind: solph.Flow()},
            loss_rate= storage.loss_rate.iloc[0],
            initial_storage_level=0,
            inflow_conversion_factor= storage.η_in.iloc[0],
            outflow_conversion_factor= storage.η_out.iloc[0],
            investment=solph.Investment(
                ep_costs=economics.annuity(capex= storage.invest.iloc[0],n= storage.ANF.iloc[0], wacc= r)))

    #Wärmespeichersystem
th_l_stor_ind = solph.components.GenericStorage(
        label="Waermespeicher IND",
        inputs={btes_ind: solph.Flow()},
        outputs={btes_ind: solph.Flow()},
            loss_rate= storage.loss_rate.iloc[1],
            initial_storage_level=0,
            inflow_conversion_factor= storage.η_in.iloc[1],
            outflow_conversion_factor= storage.η_out.iloc[1],
            investment=solph.Investment(
                ep_costs=economics.annuity(capex= storage.invest.iloc[1],n= storage.ANF.iloc[1], wacc= r)))

th_stor_in_ind = solph.Transformer(#Wärmeübertrager - Eingang
    label='WÜ IND Ein',
    inputs={bth_l_ind: solph.Flow()},
    outputs={btes_ind: solph.Flow(
        investment=solph.Investment(
            ep_costs=economics.annuity(capex= transformer.invest.iloc[7],n= transformer.ANF.iloc[7], wacc= r)))},
        conversion_factors={btes_ind: transformer.η_th.iloc[7]})

th_stor_out_ind = solph.Transformer(#Wärmeübertrager - Ausgang
    label='WÜ IND Aus',
    inputs={btes_ind: solph.Flow()},
    outputs={bth_l_ind: solph.Flow(
        investment=solph.Investment(
            ep_costs=economics.annuity(capex= transformer.invest.iloc[8],n= transformer.ANF.iloc[8], wacc= r)))},
        conversion_factors={bth_l_ind: transformer.η_th.iloc[8]})

h2_stor_ind = solph.components.GenericStorage(
    label="H2-Speicher IND",
    inputs={bh2_ind: solph.Flow()},
    outputs={bh2_ind: solph.Flow()},
        loss_rate= storage.loss_rate.iloc[2],
        initial_storage_level=0,
        inflow_conversion_factor= storage.η_in.iloc[2],
        outflow_conversion_factor= storage.η_in.iloc[2],
        investment=solph.Investment(
            ep_costs=economics.annuity(capex= storage.invest.iloc[2],n= storage.ANF.iloc[2], wacc= r)))

#Verbindungen von Buses
pv_in_ind = solph.Transformer(
    label='PV Eigenverbrauch',
    inputs={bel_pv_ind: solph.Flow()},
    outputs={bel_ind: solph.Flow()},
        conversion_factors={bel_ind: 1})

chp_in_ind = solph.Transformer(
    label='KWK Eigenverbrauch',
    inputs={bel_chp_ind: solph.Flow()},
    outputs={bel_ind: solph.Flow()},
        conversion_factors={bel_ind: 1})

heat_exchange_ind = solph.Transformer(
    label='Wärmeübergabe Prozesswärme',
    inputs={bhps_ind: solph.Flow()},
    outputs={bth_h_ind: solph.Flow()},
        conversion_factors={bth_h_ind: 1})

#Hinzufügen aller Elemente zum Energiesystem
Industrie.add(el_cons_ind,th_l_cons_ind,th_h_cons_ind,th_l_sink_ind,th_h_sink_ind,pv_exp_ind,chp_exp_ind,pv_ind,el_imp_ind,h2_imp_ind,th_imp_ind,
              wp_ind,elektrolyser_ind,gt_chp_ind,st_ind,pth_ind,bz_ind,m_chp_ind,el_stor_ind,th_l_stor_ind,th_stor_in_ind,th_stor_out_ind,h2_stor_ind,
              pv_in_ind,chp_in_ind,heat_exchange_ind)

#Lösen des Unternehmensmodells
om = solph.Model(Industrie)

#Lineare Lösung über den Solver
om.solve(solver="cplex", solve_kwargs={"tee":True})
results = solph.processing.results(om)

#Ausgabe der Lastgänge verschiedener Sektoren
stromprof = solph.views.node(results, "Strom Industrie")["sequences"]
pv_verkauf = solph.views.node(results, "Strom PV")["sequences"]
kwk_verkauf = solph.views.node(results, "Strom KWK")["sequences"]
waermeprof = solph.views.node(results, "NT-Waerme Industrie")["sequences"]
th_h_prof = solph.views.node(results, "Prozesswaerme Industrie")["sequences"]
h2_prof = solph.views.node(results, "H2 Industrie")["sequences"]
hps_prof = solph.views.node(results, "HD-Dampf Industrie")["sequences"]

#Ausgabe der investierten Leistungen und Kapazitäten im Modell
pv_invest = solph.views.node(results, "Strom PV")["scalars"]
el_invest = solph.views.node(results, "Strom Industrie")["scalars"]
kwk_invest = solph.views.node(results, "Strom KWK")["scalars"]
h2_invest = solph.views.node(results, "H2 Industrie")["scalars"]
th_l_invest = solph.views.node(results, "NT-Waerme Industrie")["scalars"]
th_h_invest = solph.views.node(results, "Prozesswaerme Industrie")["scalars"]
bat_invest = solph.views.node(results, "Stromspeicher IND")["scalars"]
tes_invest = solph.views.node(results, "Waermespeicher IND")["scalars"]
h2s_invest = solph.views.node(results, "H2-Speicher IND")["scalars"]
              


Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 12.10.0.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2019.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'C:\Users\jonbaars\AppData\Local\Temp\tmpnjem7aiz.cplex.log' open.
CPLEX> Problem 'C:\Users\jonbaars\AppData\Local\Temp\tmpqn0crk6o.pyomo.lp' read.
Read time = 0.92 sec. (40.41 ticks)
CPLEX> Problem name         : C:\Users\jonbaars\AppData\Local\Temp\tmpqn0crk6o.pyomo.lp
Objective sense      : Minimize
Variables            :  402980  [Nneg: 402972,  Box: 8]
Objective nonzeros   :   43816
Linear constraints   :  385447  [Less: 140160,  Equal: 245287]
  Nonzeros           : 1064490
  RHS nonzeros       :   24683

Variables            : Min LB: 0.0000000        Max UB: 20000.00       
Objective n

In [15]:
kwk_invest

((BHKW IND, Strom KWK), invest)                 0.000000
((Brennstoffzelle IND, Strom KWK), invest)      0.000000
((Dampfturbine IND, Strom KWK), invest)        89.871067
((GT-KWK IND, Strom KWK), invest)             575.464903
Name: 2030-01-01 00:00:00, dtype: float64